<a href="https://colab.research.google.com/github/Corona-Locator-Nederland/corona-locator-nederland/blob/main/Corona_locator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from urllib.request import urlopen
from urllib.request import urlretrieve
import pandas as pd
import numpy as np
import json
import datetime
import requests
import json
import math
import os
import seaborn as sns
import glob

In [22]:
rollover = '15-30'
today = datetime.date.today().strftime(f'%Y-%m-%d@{rollover}.csv') 

if datetime.datetime.now().strftime('%Y-%m-%d@%H-%M.csv') >= today:
  current = today
else:
  current = (datetime.date.today() + datetime.timedelta(days=1)).strftime(f'%Y-%m-%d@{rollover}.csv')

os.makedirs('downloads', exist_ok = True)
current = os.path.join('downloads', current)
if not os.path.exists(current):
  urlretrieve('https://data.rivm.nl/covid-19/COVID-19_casus_landelijk.csv', current)

for f in glob.glob('downloads/*.csv'):
  if f != current:
    os.remove(f)
covid = pd.read_csv(current, sep=';', header=0 )

In [3]:
# https://www.cbs.nl/nl-nl/onze-diensten/open-data/open-data-v4/snelstartgids-odata-v4
def get_odata(target_url):
  data = pd.DataFrame()
  while target_url:
    r = requests.get(target_url).json()
    data = data.append(pd.DataFrame(r['value']))
        
    if '@odata.nextLink' in r:
      target_url = r['@odata.nextLink']
    else:
      target_url = None          
  return data

def roundup(x):
  return int(math.ceil(x / 10.0)) * 10
def rounddown(x):
  return int(math.floor(x / 10.0)) * 10

cbs = 'https://opendata.cbs.nl/ODataApi/OData/83482NED'

leeftijden = get_odata(cbs + "/Leeftijd?$select=Key, Title&$filter=CategoryGroupID eq 3")
leeftijden.set_index('Key', inplace=True)
# zet de Title om naar begin-eind paar
leeftijden_range = leeftijden['Title'].replace(r'^(\d+) tot (\d+) jaar$', r'\1-\2', regex=True).replace(r'^(\d+) jaar of ouder$', r'\1-1000', regex=True)
# splits die paren in van-tot
leeftijden_range = leeftijden_range.str.split('-', expand=True).astype(int)
# rond the "van" naar beneden op tientallen, "tot" naar boven op tientallen, en knip af naar "90+" om de ranges uit de covid tabel te matchen
leeftijden_range[0] = leeftijden_range[0].apply(lambda x: rounddown(x)).apply(lambda x: str(x) if x < 90 else '90')
leeftijden_range[1] = (leeftijden_range[1].apply(lambda x: roundup(x)) - 1).apply(lambda x: f'-{x}' if x < 90 else '+')
# en plak ze aan elkaar
leeftijden['Range'] = leeftijden_range[0] + leeftijden_range[1]
del leeftijden['Title']

def query(f):
  if f == 'Leeftijd':
    # alle leeftijds categerien zoals hierboven opgehaald
    return '(' + ' or '.join([f"{f} eq '{k}'" for k in leeftijden.index.values]) + ')'
  if f in ['Geslacht', 'Migratieachtergrond', 'Generatie']:
    # pak hier de key die overeenkomt met "totaal"
    ids = get_odata(cbs + '/' + f)
    return f + " eq '" + ids[ids['Title'].str.contains("totaal", na=False, case=False)]['Key'].values[0] + "'"
  if f == 'Perioden':
    # voor perioden pak de laatste
    return f + " eq '" + get_odata(cbs + '/Perioden').iloc[[-1]]['Key'].values[0] + "'"
  raise ValueError(f)
# haal alle properties op waar op kan worden gefiltered en stel de query samen
filter = get_odata(cbs + '/DataProperties')
filter = ' and '.join([query(f) for f in filter[filter.Type != 'Topic']['Key'].values])

bevolking = get_odata(cbs + f"/TypedDataSet?$top=100&$filter={filter}&$select=Leeftijd, BevolkingOpDeEersteVanDeMaand_1")
# die _1 betekent waarschijnlijk dat het gedrag ooit gewijzigd is en er een nieuwe "versie" van die kolom is gepubliceerd
bevolking.rename(columns = {'BevolkingOpDeEersteVanDeMaand_1': 'BevolkingOpDeEersteVanDeMaand'}, inplace = True)
bevolking = bevolking.merge(leeftijden, left_on = 'Leeftijd', right_index = True)
# optellen om de leeftijds categorien bij elkaar te vegen om de "agegroups" uit "covid" te matchen 
bevolking = bevolking.groupby('Range')['BevolkingOpDeEersteVanDeMaand'].sum().to_frame()
bevolking['per 100k'] = 100000 / bevolking['BevolkingOpDeEersteVanDeMaand']
bevolking

,BevolkingOpDeEersteVanDeMaand,per 100k
Range,,
0-9,1760648,0.056797
10-19,1988987,0.050277
20-29,2238892,0.044665
30-39,2174964,0.045978
40-49,2172032,0.046040
50-59,2546244,0.039274
60-69,2138705,0.046757
70-79,1610080,0.062109
80-89,706450,0.141553


In [9]:
# vervang <50 en Unknown door Onbekend
covid['Cohort'] = covid['Agegroup'].replace({'<50': 'Onbekend', 'Unknown': 'Onbekend'})
# aangenomen 'gemiddelde' leeftijd van een cohort: minimum waarde + 5
assumed_cohort_age = [(cohort, [int(n) for n in cohort.replace('+', '').split('-')]) for cohort in covid['Cohort'].unique() if cohort[0].isdigit()]
assumed_cohort_age = { cohort: min(rng) + 5 for cohort, rng in assumed_cohort_age }
covid['~Leeftijd'] = covid['Cohort'].apply(lambda x: assumed_cohort_age.get(x, np.nan))

# vervang Yes door 1, No/Unkown door 0 zodat een simpele sum ons straks het totaal geeft
for src, tgt in {'Hospital_admission': 'Hospital_admission_int', 'Deceased': 'Deceased_int'}.items():
  covid[tgt] = covid[src].replace({'Yes': 1, 'No': 0, 'Unknown': np.nan})

# verwijder tijd
covid['Date_file_date'] = pd.to_datetime(covid['Date_file'].replace(r' .*', '', regex=True))

covid['Date_statistics_date'] = pd.to_datetime(covid['Date_statistics'])

# weken terug = verschil tussen Date_file en Date_statistcs, gedeeld door 7 dagen
covid['Weken terug'] = np.floor((covid['Date_file_date'] - covid['Date_statistics_date'])/np.timedelta64(7, 'D')).astype(np.int)

# voeg kleurnummer en totaal toe
def color_and_total(df, prefix, total):
  columns = list(bevolking.index) + ['Onbekend']
  # insert missing cohorts
  for col in columns:
    if not col in df:
      df[col] = np.nan
  # make sure the columns are ordered according to the cohorts, so the rename is safe
  df = df[columns]
  # geef de kolom een prefix om ze uniek te houden
  df.columns = [prefix + col for col in columns]

  # sommeer over de cohorten om een totaal te krijgen 
  df[total] = df.sum(axis=1)

  # voeg de kleur kolommen toe
  for col in columns:
    df[prefix + 'c' + col] = ((df[prefix + col] / df[[prefix + col for col in columns]].max(axis=1)) * 1000).fillna(0).astype(int)

  # voeg de "per honderdduizend" kolommen toe
  for col in columns:
    per100k = bevolking.loc[col]['per 100k'] if col in bevolking.index else np.nan
    df[prefix + '100k' + col] = df[prefix + col] * per100k
  
  # en daarvoor ook kleur kolommen
  for col in columns:

    df[prefix + '100k' + 'c' + col] = ((df[prefix + '100k' + col] / df[[prefix + '100k' + col for col in columns]].max(axis=1)) * 1000).fillna(0).astype(int)

  # herschikken van de kolommen om totaal vooraan te krijgen
  return df[[total] + [col for col in df if col != total]]

# registraties is een count per cohort
registraties = covid.groupby(['Weken terug', 'Cohort'])['Date_file_date'].count().unstack(fill_value=np.nan).reset_index().rename_axis(None, axis=1)
registraties.set_index('Weken terug', inplace=True)
registraties = color_and_total(registraties, 'r', 'Totaal registraties')

# hospitalised is de som van 'Yes' voor Hospital_admission per cohort
hospitalised = covid.groupby(['Weken terug', 'Cohort'])['Hospital_admission_int'].sum().unstack(fill_value=np.nan).reset_index().rename_axis(None, axis=1)
hospitalised.set_index('Weken terug', inplace=True)
hospitalised = color_and_total(hospitalised, 'h', 'Totaal hospitalised')

# deceased is de som van 'Yes' voor Deceased per cohort
deceased = covid.groupby(['Weken terug', 'Cohort'])['Deceased_int'].sum().unstack(fill_value=np.nan).reset_index().rename_axis(None, axis=1)
deceased.set_index('Weken terug', inplace=True)
deceased = color_and_total(deceased, 'd', 'Totaal deceased')

# de totale tabel is keyed op 'Weken terug', en we berekenen hier direct de mean aangenomen leeftijd per weken-terug van de registraties
stats = covid.groupby(['Weken terug'])['~Leeftijd'].mean().to_frame()
# hernoemen kolom
stats.columns = ['Gemiddelde leeftijd']

# Datum is de Date_file
stats['Datum'] = Date_file = covid['Date_file_date'].unique()[0]
# terug naar een datetime
Date_file = Date_file.astype('M8[D]').astype('O')
# periode afgeleid van weken-terug (= de index voor deze dataframe)
stats['Periode'] = stats.index.to_series().apply(lambda x: (Date_file + datetime.timedelta(weeks=-(x+1), days=1)).strftime('%d/%m') + ' - ' + (Date_file + datetime.timedelta(weeks=-x)).strftime('%d/%m'))
# herschikken
stats = stats[['Datum', 'Periode', 'Gemiddelde leeftijd']]
# toevoegen van registraties, hospitalized en deceased 
stats = stats.join(registraties).join(hospitalised).join(deceased)
print(stats.head())
cm = sns.light_palette("green", as_cmap=True)
registraties[[col for col in registraties.columns if col[0] == 'r' and col[1] != 'c' and not col.startswith('r100')]].style.background_gradient(cmap=cm, axis=1)

                 Datum        Periode  ...  d100kc90+  d100kcOnbekend
Weken terug                            ...                           
0           2021-01-02  27/12 - 02/01  ...       1000               0
1           2021-01-02  20/12 - 26/12  ...       1000               0
2           2021-01-02  13/12 - 19/12  ...       1000               0
3           2021-01-02  06/12 - 12/12  ...       1000               0
4           2021-01-02  29/11 - 05/12  ...       1000               0

[5 rows x 138 columns]


,r0-9,r10-19,r20-29,r30-39,r40-49,r50-59,r60-69,r70-79,r80-89,r90+,rOnbekend
Weken terug,,,,,,,,,,,
0,483.000000,3519.000000,5360.000000,4120.000000,4253.000000,5366.000000,3355.000000,2324.000000,1600.000000,608.000000,nan
1,1019.000000,7670.000000,9540.000000,8029.000000,9378.000000,10915.000000,6618.000000,4061.000000,2789.000000,945.000000,3.000000
2,1241.000000,11693.000000,11818.000000,10326.000000,12487.000000,13626.000000,8118.000000,4834.000000,3034.000000,1005.000000,4.000000
3,1190.000000,11467.000000,9276.000000,8526.000000,10092.000000,10585.000000,6395.000000,3843.000000,2383.000000,850.000000,5.000000
4,743.000000,8055.000000,6838.000000,5947.000000,7100.000000,7439.000000,4430.000000,2564.000000,1594.000000,543.000000,1.000000
5,511.000000,5775.000000,5138.000000,4693.000000,5287.000000,5643.000000,3162.000000,1950.000000,1214.000000,433.000000,3.000000
6,421.000000,5770.000000,5645.000000,5096.000000,5702.000000,6040.000000,3359.000000,2065.000000,1414.000000,483.000000,1.000000
7,405.000000,4838.000000,6055.000000,5319.000000,5772.000000,6458.000000,3879.000000,2190.000000,1537.000000,530.000000,1.000000
8,350.000000,4706.000000,6974.000000,5809.000000,6077.000000,7007.000000,4241.000000,2363.000000,1796.000000,578.000000,2.000000


In [5]:
# publish
if 'GITHUB_TOKEN' in os.environ:
  print('Publishing to', os.environ['GITHUB_REPOSITORY'])
  today = datetime.date.today().strftime('%Y-%m-%d') + '.csv'
  latest = 'covid.csv'
  stats.to_csv(latest)

  import github3 as github
  gh = github.GitHub(token=os.environ['GITHUB_TOKEN'], session=github.session.GitHubSession(default_read_timeout=60))
  repo = gh.repository(*os.environ['GITHUB_REPOSITORY'].split('/'))
  release = repo.release_from_tag('covid')
  assets = { asset.name: asset for asset in release.assets() }

  # remove existing
  for asset in [today, latest]:
    if asset in assets:
      assets[asset].delete()
    with open(latest) as f:
      release.upload_asset(asset=f, name=asset, content_type='text/csv')

# Bart's speeltuin

In [6]:
# speeltuin
